In [ ]:
!bash get_train_data.sh; bash get_train_data.sh; bash get_train_data.sh; bash get_train_data.sh; bash get_train_data.sh

In [ ]:
!bash get_train_data.sh; bash get_train_data.sh; bash get_train_data.sh; bash get_train_data.sh; bash get_train_data.sh

In [ ]:
!cat train-data*.csv > models/20192009_12/train.csv

In [ ]:
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
import pickle
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [ ]:
!cat models/20192009_12/train.csv | wc -l

In [ ]:
!ls models/20192009_12/

In [ ]:
features = ["f" + str(i) for i in range(0,123)]
label = ["action"]

headers = features + label

df = pd.read_csv('models/20192009_12/train.csv', 
                 sep = ',', 
                 header = None,
                 names = headers)
df

In [ ]:
print("headers length: {}, df lenght: {}".format(len(headers), len(df)))

In [ ]:
df["f0"].value_counts()

In [ ]:
print("different actions count \n{}".format(df.drop_duplicates()['action'].value_counts()))

In [ ]:
print("no of states without duplicates: {}".format(len(df.drop_duplicates())))

In [ ]:
y_train_all = df["action"]
X_train_all = df.drop(columns="action")

In [ ]:
print("no of states without duplicates and action column: {}".format(len(X_train_all.drop_duplicates())))

In [ ]:
import pandas_profiling
pandas_profiling.ProfileReport(df)

In [ ]:
# X_train_all[["f0", "f1"]]

In [ ]:
def evaluate_model(xtrain, ytrain, xtest, ytest, model):
    y_pred_test = model.predict(xtest)
    y_pred_train = model.predict(xtrain)
    test_accuracy = accuracy_score(ytest, y_pred_test)
    train_accuracy = accuracy_score(ytrain, y_pred_train)
    print('Training accuracy: {0:.2f}, evaluation accuracy: {1:.2f}'.format(train_accuracy, test_accuracy))


Basic logisting regresion

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train_all, y_train_all, test_size=0.2)

classifier = LogisticRegression().fit(X_train, y_train)

In [ ]:
evaluate_model(X_train, y_train, X_test, y_test, classifier)

In [ ]:
pickle.dump(classifier, open("ai_model_logit.pkl","wb"))

xgboost

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train_all, y_train_all, 
                                                    test_size=0.2, random_state=7)

xgb_model = XGBClassifier(n_estimators=500).fit(X_train, y_train,eval_set = [[X_test, y_test]], verbose = 1)

In [ ]:
evaluate_model(X_train, y_train, X_test, y_test, xgb_model)

In [ ]:
xgb_model = XGBClassifier(n_estimators=500).fit(X_train_all, y_train_all)

In [ ]:
pickle.dump(xgb_model, open("ai_model_xgb.pkl","wb"))

random forest model

In [ ]:
forest_model = RandomForestClassifier(random_state=1, n_estimators=100)
forest_model.fit(X_train, y_train)

In [ ]:
evaluate_model(X_train, y_train, X_test, y_test, forest_model)

In [ ]:
pickle.dump(forest_model, open("ai_model_forest.pkl","wb"))

In [ ]:
# min_max_scaler = MinMaxScaler()
# np_scaled = min_max_scaler.fit_transform(df)
# df_normalized = pd.DataFrame(np_scaled)
# df_normalized.columns = feature_cols + ['Class']

In [ ]:
# def make_crosses(X_train, X_validation, degree=2):
#   from sklearn.preprocessing import PolynomialFeatures
#   poly = PolynomialFeatures(degree=degree)
#   X_train_poly = poly.fit_transform(X_train)
#   X_validation_poly = poly.transform(X_validation)
#   return X_train_poly, X_validation_poly

In [ ]:
mlp_model = MLPClassifier(solver='adam', alpha=1e-5, 
                    hidden_layer_sizes=(60, 24, 24), 
                          random_state=1, verbose=True)

mlp_model.fit(X_train, y_train)

In [ ]:
evaluate_model(X_train, y_train, X_test, y_test, mlp_model)

In [ ]:
mlp_model = MLPClassifier(solver='adam', alpha=1e-5, 
                    hidden_layer_sizes=(24, 24, 24), 
                          random_state=1, verbose=True, max_iter=1000)

mlp_model.fit(X_train_all, y_train_all)

Save models to pkl

In [ ]:
pickle.dump(mlp_model, open("ai_model_mlp.pkl","wb"))

In [ ]:
!bash evaluate-ai.sh

In [ ]:
# model_p = pickle.load(open("ai_model.pkl","rb"))
game_state_4f = (np.array([8,280,630,-1,1,674,-3,7,667,-1,2,649,-3,2,662,0,3,662,-1,4,655,-1,5,608,2,7,1,8,288,630,-1,1,672,-3,7,668,-1,2,650,-3,2,661,0,3,657,-1,4,649,-1,5,602,2,7,1,8,296,629,-1,1,670,-3,7,669,-1,2,651,-3,2,660,0,3,651,-1,4,643,-1,5,596,2,7,1,8,296,628,-1,1,665,-3,7,667,-1,2,650,-3,2,658,0,3,647,-1,4,638,-1,5,589,2,7])
                 .reshape(1, -1))

game_state_1f = (np.array([25, -410, 308, 5, 1, -343, 483, 1, 2, -417, 592, 5, 2, -184, 589, 1, 2, -222, 275, 7, 2, -93, 143, 3, -2, -128, 549, 1, -3, -121, 399, 6, -3]).reshape(1, -1))

xgb_model.predict(game_state_1f)